# Remote Interactive Task Manager LSASS Dump

## Playbook Tags

**ID:** WINEXEC191030201010

**Author:** Roberto Rodriguez [@Cyb3rWard0g](https://twitter.com/Cyb3rWard0g)

**References:**

## ATT&CK Tags

**Tactic:** Credential Access

**Technique:** Credential Dumping

## Applies To

## Technical Description

The Windows Task Manager may be used to dump the memory space of lsass.exe to disk for processing with a credential access tool such as Mimikatz. This is performed by launching Task Manager as a privileged user, selecting lsass.exe, and clicking “Create dump file”. This saves a dump file to disk with a deterministic name that includes the name of the process being dumped. [Reference](https://car.mitre.org/analytics/CAR-2019-08-001/)

### Additional Reading


## Permission Required

Administrator

## Hypothesis

Adversaries might be RDPing to computers in my environment and interactively dumping the memory contents of LSASS with task manager.

## Attack Simulation Dataset

| Environment| Name | Description |
|--------|---------|---------|
| [Shire](https://github.com/Cyb3rWard0g/mordor/tree/acf9f6be6a386783a20139ceb2faf8146378d603/environment/shire) | [remoteinteractive_taskmngr_lsass_dump](https://github.com/hunters-forge/mordor/blob/master/small_datasets/windows/credential_access/credential_dumping_T1003/remoteinteractive_taskmngr_lsass_dump.md) | An adversary with local admin rights over the host can easily RDP to a victims box, use task manager interactively and dump the memory space of lsass. |
| [Shire](https://github.com/Cyb3rWard0g/mordor/tree/acf9f6be6a386783a20139ceb2faf8146378d603/environment/shire) | [interactive_taskmngr_lsass_dump](https://github.com/hunters-forge/mordor/blob/master/small_datasets/windows/credential_access/credential_dumping_T1003/interactive_taskmngr_lsass_dump.md) | An adversary with local admin rights over the host can easily use task manager interactively and dump the memory space of lsass. |

## Recommended Data Sources

| Event ID | Event Name | Log Provider | Audit Category | Audit Sub-Category | ATT&CK Data Source |
|---------|---------|----------|----------|---------|---------|
| [4778](https://github.com/hunters-forge/OSSEM/blob/master/data_dictionaries/windows/security/events/event-4778.md) | A session was reconnected to a Windows Station | Microsoft-Windows-Security-Auditing | | | Windows Event Logs |
| [1](https://github.com/Cyb3rWard0g/OSSEM/blob/master/data_dictionaries/windows/sysmon/event-1.md) | Process Creation | Microsoft-Windows-Sysmon | | | Process Monitoring |
| [10](https://github.com/hunters-forge/OSSEM/blob/master/data_dictionaries/windows/sysmon/event-10.md) | Process Access | Microsoft-Windows-Sysmon | |  | Process Monitoring |
| [11](https://github.com/hunters-forge/OSSEM/blob/master/data_dictionaries/windows/sysmon/event-11.md) | File Create | Microsoft-Windows-Sysmon |  |  | File Monitoring |

## Data Analytics
### Initialize Analytics Engine

In [1]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession

In [2]:
win = winlogbeat()
spark = SparkSession.builder.appName("Mordor").config("spark.sql.caseSensitive", "True").getOrCreate()
print(spark)

#### Prepare & Process Mordor File

In [3]:
mordor_file = win.extract_nested_fields("mordor/small_datasets/remoteinteractive_taskmngr_lsass_dump_2019-10-27055035.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 7..
[+] DataFrame Returned !


#### Register Mordor DataFrame as a SQL temporary view

In [4]:
mordor_file.createOrReplaceTempView("mordor_file")

### Validate Analytic I
| FP Rate | Source | Analytic Logic | Description |
|--------|---------|---------|---------|
| Medium | Security | SELECT `@timestamp`, computer_name, Image, TargetFilename, ProcessGuid FROM mordor_file WHERE channel = "Microsoft-Windows-Sysmon/Operational" AND event_id = 11 AND Image LIKE "%taskmgr.exe" AND lower(TargetFilename) RLIKE ".*lsass.*\.dmp" | Look for taskmgr creating files which name contains the string lsass and with extension .dmp. |

In [5]:
sysmon_file_df = spark.sql(
    '''
    SELECT `@timestamp`, computer_name, Image, TargetFilename, ProcessGuid
    FROM mordor_file
    WHERE channel = "Microsoft-Windows-Sysmon/Operational"
        AND event_id = 11
        AND Image LIKE "%taskmgr.exe"
        AND lower(TargetFilename) RLIKE ".*lsass.*\.dmp"
    '''
)
sysmon_file_df.show(10, False)

+------------------------+----------------+-------------------------------+------------------------------------------------+--------------------------------------+
|@timestamp              |computer_name   |Image                          |TargetFilename                                  |ProcessGuid                           |
+------------------------+----------------+-------------------------------+------------------------------------------------+--------------------------------------+
|2019-10-27T05:51:07.428Z|HFDC01.shire.com|C:\Windows\system32\taskmgr.exe|C:\Users\ADMINI~1\AppData\Local\Temp\2\lsass.DMP|{e7604876-3040-5db5-0000-00100e2e8300}|
+------------------------+----------------+-------------------------------+------------------------------------------------+--------------------------------------+



### Validate Analytic II
| FP Rate | Source | Analytic Logic | Description |
|--------|---------|---------|---------|
| Medium | Sysmon | SELECT `@timestamp`, computer_name, SourceImage, TargetImage, GrantedAccess FROM mordor_file WHERE channel = "Microsoft-Windows-Sysmon/Operational" AND event_id = 10 AND lower(SourceImage) LIKE "%taskmgr.exe" AND lower(TargetImage) LIKE "%lsass.exe" AND (lower(CallTrace) RLIKE ".\*dbgcore\.dll.\*" OR lower(CallTrace) RLIKE ".\*dbghelp\.dll.\*") | Look for task manager access lsass and with functions from dbgcore.dll or dbghelp.dll libraries |

In [6]:
sysmon_process_access_df = spark.sql(
    '''
    SELECT `@timestamp`, computer_name, SourceImage, TargetImage, GrantedAccess
    FROM mordor_file
    WHERE channel = "Microsoft-Windows-Sysmon/Operational"
        AND event_id = 10
        AND lower(SourceImage) LIKE "%taskmgr.exe"
        AND lower(TargetImage) LIKE "%lsass.exe"
        AND (lower(CallTrace) RLIKE ".*dbgcore\.dll.*" OR lower(CallTrace) RLIKE ".*dbghelp\.dll.*")
    '''
)
sysmon_process_access_df.show(10,False)

+------------------------+----------------+-------------------------------+-----------------------------+-------------+
|@timestamp              |computer_name   |SourceImage                    |TargetImage                  |GrantedAccess|
+------------------------+----------------+-------------------------------+-----------------------------+-------------+
|2019-10-27T05:51:07.490Z|HFDC01.shire.com|C:\Windows\system32\taskmgr.exe|C:\Windows\system32\lsass.exe|0x1fffff     |
+------------------------+----------------+-------------------------------+-----------------------------+-------------+



### Validate Analytic III
| FP Rate | Source | Analytic Logic | Description |
|--------|---------|---------|---------|
| Medium | Sysmon | SELECT `@timestamp`, computer_name, SourceImage, TargetImage, GrantedAccess FROM mordor_file WHERE channel = "Microsoft-Windows-Sysmon/Operational" AND event_id = 10 AND lower(TargetImage) LIKE "%lsass.exe" AND (lower(CallTrace) RLIKE ".\*dbgcore\.dll.\*" OR lower(CallTrace) RLIKE ".\*dbghelp\.dll.\*") | Look for any process accessing lsass and with functions from dbgcore.dll or dbghelp.dll libraries |

In [7]:
sysmon_process_access_df = spark.sql(
    '''
    SELECT `@timestamp`, computer_name, SourceImage, TargetImage, GrantedAccess
    FROM mordor_file
    WHERE channel = "Microsoft-Windows-Sysmon/Operational"
        AND event_id = 10
        AND lower(TargetImage) LIKE "%lsass.exe"
        AND (lower(CallTrace) RLIKE ".*dbgcore\.dll.*" OR lower(CallTrace) RLIKE ".*dbghelp\.dll.*")
    '''
)
sysmon_process_access_df.show(10,False)

+------------------------+----------------+-------------------------------+-----------------------------+-------------+
|@timestamp              |computer_name   |SourceImage                    |TargetImage                  |GrantedAccess|
+------------------------+----------------+-------------------------------+-----------------------------+-------------+
|2019-10-27T05:51:07.490Z|HFDC01.shire.com|C:\Windows\system32\taskmgr.exe|C:\Windows\system32\lsass.exe|0x1fffff     |
+------------------------+----------------+-------------------------------+-----------------------------+-------------+



### Validate Analytic IV
| FP Rate | Source | Analytic Logic | Description |
|--------|---------|---------|---------|
| Low | Security | SELECT o.`@timestamp`, o.computer_name, o.Image, o.ProcessGuid, a.SourceProcessGUID, o.CommandLine FROM mordor_file o INNER JOIN (SELECT computer_name,SourceProcessGUID FROM mordor_file WHERE channel = "Microsoft-Windows-Sysmon/Operational" AND event_id = 10 AND lower(TargetImage) LIKE "%lsass.exe" AND (lower(CallTrace) RLIKE ".*dbgcore\.dll.*" OR lower(CallTrace) RLIKE ".*dbghelp\.dll.*")) a ON o.ProcessGuid = a.SourceProcessGUID WHERE o.channel = "Microsoft-Windows-Sysmon/Operational" AND o.event_id = 1 | Look for combinations of process access and process creation to get more context around potential lsass dump form task manager or other binaries |

In [8]:
sysmon_1_10 = spark.sql(
    '''
    SELECT o.`@timestamp`, o.computer_name, o.Image, o.LogonId, o.ProcessGuid, a.SourceProcessGUID, o.CommandLine
    FROM mordor_file o
    INNER JOIN (
        SELECT computer_name,SourceProcessGUID
        FROM mordor_file
        WHERE channel = "Microsoft-Windows-Sysmon/Operational"
            AND event_id = 10
            AND lower(TargetImage) LIKE "%lsass.exe"
            AND (lower(CallTrace) RLIKE ".*dbgcore\.dll.*" OR lower(CallTrace) RLIKE ".*dbghelp\.dll.*")
        ) a
    ON o.ProcessGuid = a.SourceProcessGUID
    WHERE o.channel = "Microsoft-Windows-Sysmon/Operational"
        AND o.event_id = 1
    '''
)
sysmon_1_10.show(10,False)

+------------------------+----------------+-------------------------------+--------+--------------------------------------+--------------------------------------+------------------------------------+
|@timestamp              |computer_name   |Image                          |LogonId |ProcessGuid                           |SourceProcessGUID                     |CommandLine                         |
+------------------------+----------------+-------------------------------+--------+--------------------------------------+--------------------------------------+------------------------------------+
|2019-10-27T05:50:56.347Z|HFDC01.shire.com|C:\Windows\System32\Taskmgr.exe|0x6a7c4d|{e7604876-3040-5db5-0000-00100e2e8300}|{e7604876-3040-5db5-0000-00100e2e8300}|"C:\Windows\system32\taskmgr.exe" /7|
+------------------------+----------------+-------------------------------+--------+--------------------------------------+--------------------------------------+------------------------------------+


### Validate Analytic V
| FP Rate | Source | Analytic Logic | Description |
|--------|---------|---------|---------|
| Low | Security | SELECT o.`@timestamp`, o.computer_name, o.SessionName, o.AccountName, o.ClientName, o.ClientAddress, a.Image, a.CommandLine FROM mordor_file o INNER JOIN (SELECT LogonId, Image, CommandLine FROM sysmon_join) a ON o.LogonID = a.LogonId WHERE lower(o.channel) = "security" AND o.event_id = 4778| Look for binaries accessing lsass that are running under the same logon context of a user over an RDP session |

In [15]:
sysmon_1_10.createOrReplaceTempView("sysmon_join")

In [25]:
sysmon_security = spark.sql(
    '''
    SELECT o.`@timestamp`, o.computer_name, o.SessionName, o.AccountName, o.ClientName, o.ClientAddress, a.Image, a.CommandLine
    FROM mordor_file o
    INNER JOIN (
        SELECT LogonId, Image, CommandLine
        FROM sysmon_join
        ) a
    ON o.LogonID = a.LogonId
    WHERE lower(o.channel) = "security"
        AND o.event_id = 4778
    '''
)
sysmon_security.show(10,False)

+------------------------+----------------+-----------+-------------+----------+-------------+-------------------------------+------------------------------------+
|@timestamp              |computer_name   |SessionName|AccountName  |ClientName|ClientAddress|Image                          |CommandLine                         |
+------------------------+----------------+-----------+-------------+----------+-------------+-------------------------------+------------------------------------+
|2019-10-27T05:50:47.424Z|HFDC01.shire.com|RDP-Tcp#5  |Administrator|IT001     |172.18.39.105|C:\Windows\System32\Taskmgr.exe|"C:\Windows\system32\taskmgr.exe" /7|
+------------------------+----------------+-----------+-------------+----------+-------------+-------------------------------+------------------------------------+



## Detection Blind Spots

## Hunter Notes

* Add context to your queries by joining RDP remote interactive authentication events by the logon ID.

## Hunt Output


## References

* https://car.mitre.org/analytics/CAR-2019-08-001/